# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- template script for analyzing modes spectra for sumatra

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from obspy import read
from obspy.core import UTCDateTime
from obspy.geodetics import gps2dist_azimuth
from obspy.geodetics import kilometers2degrees

from pysep import Pysep

from lib_seis import get_dist_az
from lib_seis import plot_event_station
from lib_seis import station_info_list
from lib_seis import wf_fft
from lib_seis import w2fstack

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

origin time = 2004-12-26 00:58:53 \
latitude    = 3.09 N \
longitude   = 94.26 E \
depth       = 30 km \
magnitude   = 9.1

In [ ]:
# setting event and data parameters

origin_time                  = UTCDateTime('2004,12,26,00,58,53')
event_latitude               = 3.09
event_longitude              = 94.26
event_depth_km               = 30
event_magnitude              = 9.1
resample_freq                = 1
data_dir                     = 'datawf/sumatra_LHZ_acc/SAC'

In [ ]:
# read downloaded raw LHZ data

st       = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# trimming details for waveforms needing it
# ID | starttime | endtime

trim_details = [['IU.QSPA.20.LHZ', 0       , 4.4800e5],
                ['II.ABKT.00.LHZ', 0       , 6.7410e5],
                ['G.ATD.20.LHZ'  , 0       , 2.8400e5],
                ['IU.BILL.20.LHZ', 0       , 4.3050e5],
                ['II.BORG.20.LHZ', 0       , 3.1700e5],
                ['IU.CHTO.20.LHZ', 0       , 3.6270e5],
                ['II.EFI.20.LHZ' , 0       , 2.7500e5],
                ['IU.GUMO.20.LHZ', 0       , 1.7980e5],
                ['IU.HRV.20.LHZ' , 0       , 4.5800e5],
                ['IU.MIDW.20.LHZ', 0.7500e4, 9.0720e5],
                ['II.PFO.20.LHZ' , 0       , 3.3205e5],
                ['G.SCZ.20.LHZ'  , 0       , 1.9300e5],
                ['G.TAM.20.LHZ'  , 0       , 5.5800e5],
                ['IU.TIXI.20.LHZ', 0       , 6.6150e5],
                ['G.WUS.20.LHZ'  , 0       , 5.8200e5]]

In [ ]:
# listing selected waveforms

picked_waveforms = [['G' ,'CAN' ,''  ,'LHZ'],
                    ['G' ,'CAN' ,''  ,'LHZ'],
                    ['G' ,'CAN' ,''  ,'LHZ']]

##############################################
####### PASTE COMPLETE LIST ABOVE ############
##############################################

In [ ]:
# storing station information for selected stations
station_lats, station_lons, station_tags = station_info_list(st, picked_waveforms)

In [ ]:
print('\nSource receiver map')    
plot_event_station(event_latitude,event_longitude,slat=station_lats,slon=station_lons,stas=station_tags)

In [ ]:
print('Table of station azimuths and distances\n')
distance_deg, azimuth_deg, distance_km = get_dist_az(event_longitude,event_latitude,station_lons,station_lats,station_tags)#

In [ ]:
# computing and plotting the discrete Fourier transforms for the selected waveforms

amps  = []
freqs = []

Nyquist_frequency = resample_freq/2

for i, waveform_id in enumerate(picked_waveforms):
    
    ID = f'{waveform_id[0]}.{waveform_id[1]}.{waveform_id[2]}.{waveform_id[3]}'
    tr = st.select(id=ID)
        
    if tr[0].id in trim_details[:][0]:
        start_time = tr[0].stats.starttime + trim_details[trim_details[:][0].index(tr[0].id)][1] 
        end_time   = tr[0].stats.starttime + trim_details[trim_details[:][0].index(tr[0].id)][2]
        tr[0].trim(start_time,end_time)

    tr.detrend('constant')
    tr.detrend('linear')
    tr.taper(max_percentage=0.2, type="cosine")
    fft_amp, fft_phase, fft_freq = wf_fft(tr[0].data, Nyquist_frequency)

    freqs.append(fft_freq)
    amps.append(fft_amp)

In [ ]:
# plotting amplitude spectra for selected waveforms

for i, waveform_id in enumerate(picked_waveforms):    
    
    ID = f'{waveform_id[0]}.{waveform_id[1]}.{waveform_id[2]}.{waveform_id[3]}'
    
    fft_freq = freqs[i]
    fft_amp  = amps[i]
    
    plt.figure()
    plt.plot(freqs[i]*1E3,amps[i])
    plt.xlim(0.2,1)
    #plt.ylim(0,40)
    xmask = np.logical_and(freqs[i]*1e3 > .2, freqs[i]*1e3 < 1)
    plt.ylim(0,np.max(amps[i][xmask])*1.1)
    plt.title(ID)
    plt.xlabel(f'Frequency (mHz)')
    plt.ylabel(f'Amplitude (counts s)')
    plt.show()

In [ ]:
# example of stacking spectra

f1 = 0.2 * 1E-3
f2 = 10  * 1E-3
n  = 100000

amp_stack, freq, amplitudes = w2fstack(freqs,amps,f1,f2,n)                                                     

plt.figure()
plt.plot(freq*1E3, amp_stack, 'b') 
plt.title('Stacked of amplitude spectra of selected waveforms')
plt.xlabel('Frequency [mHz]')
plt.ylabel('Amplitude ')
plt.show()